[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/function-calling/anthropic/weaviate-query-agent-tool.ipynb)

## Weaviate Query Agent with Anthropic

This notebook will show you how to define the Weaviate Query Agent as a tool through Anthropic.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install the Anthropic Python API with `pip install anthropic`. We're using version `0.49.0` at the time of creating this notebook.
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [ ]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

import anthropic

In [1]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"] = ""

### Define Query Agent function

In [ ]:
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={ # add the API key to the model provider from your Weaviate collection, for example `headers={"X-VoyageAI-Api-Key": os.getenv("VOYAGE_API_KEY")}` 
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
    return query_agent.run(query).final_answer

### Connect to Anthropic client and define Query Agent function

In [8]:
def run_assistant(message, chat_history=None):
    if chat_history is None:
        chat_history = []

    # Initialize Anthropic client
    anthropic_client = anthropic.Anthropic(os.getenv("ANTHROPIC_API_KEY")
    )

    # Print user message
    print(f"Question:\n{message}")
    print("="*50)

    # Initial message to Claude
    response = anthropic_client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        tools=[
            {
                "name": "send_query_agent_request",
                "description": "Send a query to a database filled with information about Weaviate and get the response",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The question or query to search for in the database. This can be any natural language question related to the content stored in the database.",
                        }
                    },
                    "required": ["query"],
                },
            }
        ],
        messages=[*chat_history, {"role": "user", "content": message}]
    )

    while response.stop_reason == 'tool_use':
        # Extract tool use blocks from the response
        tool_blocks = [block for block in response.content if block.type == 'tool_use']
        
        if any(block.type == 'text' for block in response.content):
            print("Tool plan:")
            print([block.text for block in response.content if block.type == 'text'][0], "\n")
            
        print("Tool calls:")
        for tool_block in tool_blocks:
            print(f"Tool name: {tool_block.name} | Parameters: {tool_block.input}")
        print("="*50)
        
        # Execute tool calls and collect results
        tool_results = []
        for tool_block in tool_blocks:
            try:
                tool_output = send_query_agent_request(**tool_block.input)
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_block.id,
                    "content": tool_output
                })
            except Exception as e:
                print(f"Error executing tool {tool_block.name}: {str(e)}")
                tool_results.append({
                    "type": "tool_result", 
                    "tool_use_id": tool_block.id,
                    "content": f"Error: {str(e)}"
                })

        # Get next response from Claude with tool outputs
        response = anthropic_client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=1024,
            messages=[
                *chat_history,
                {"role": "user", "content": message},
                {"role": "assistant", "content": response.content},
                {"role": "user", "content": tool_results}
            ]
        )

    # Print final response
    print("Final response:")
    final_text = next((block.text for block in response.content if block.type == 'text'), None)
    print(final_text)
    print("="*50)
    
    # Update chat history with final exchange
    chat_history.extend([
        {"role": "user", "content": message},
        {"role": "assistant", "content": response.content}
    ])
    
    return chat_history


### Query time

In [9]:
chat_history = run_assistant("How do I run Weaviate with Docker?")

Question:
How do I run Weaviate with Docker?
Tool plan:
I'll help you find information about running Weaviate with Docker. Let me search for that information for you. 

Tool calls:
Tool name: send_query_agent_request | Parameters: {'query': 'How to run Weaviate with Docker'}
Final response:
# Running Weaviate with Docker

Here's how to run Weaviate using Docker:

## Prerequisites
- Docker and Docker Compose installed on your system
- Basic familiarity with Docker commands

## Step-by-Step Instructions

1. **Install Docker and Docker Compose**
   - For Windows/Mac: Install [Docker Desktop](https://www.docker.com/products/docker-desktop)
   - For Linux: Install [Docker Engine](https://docs.docker.com/engine/install/) and [Docker Compose](https://docs.docker.com/compose/install/)

2. **Get the Docker Compose Configuration**
   - Option 1: Use the [Weaviate configuration tool](https://weaviate.io/developers/weaviate/installation/docker-compose) to generate a custom `docker-compose.yml`
   